In [29]:
import pandas as pd
import os
import csv

In [30]:
# Define the MP, Party, and Constituency Classes
class MP:
    def __init__(self, first_name, last_name, gender, result, party, votes_cast):
        self.first_name = first_name
        self.last_name = last_name
        self.gender = gender
        self.result = result
        self.party = party
        self.votes_cast = votes_cast

class Party:
    def __init__(self, name):
        self.name = name
        self.total_votes = 0
        self.total_candidates = 0

    def add_votes(self, votes):
        self.total_votes += votes
        self.total_candidates += 1

    def get_percentage(self, total_votes):
        return (self.total_votes / total_votes) * 100 if total_votes > 0 else 0

class Constituency:
    def __init__(self, name, region, country, electorate, valid_votes, invalid_votes, mp):
        self.name = name
        self.region = region
        self.country = country
        self.electorate = electorate
        self.valid_votes = valid_votes
        self.invalid_votes = invalid_votes
        self.mp = mp
        self.parties = {}

    def add_party_votes(self, party_name, votes):
        if party_name not in self.parties:
            self.parties[party_name] = Party(party_name)
        self.parties[party_name].add_votes(votes)

    def get_party_percentage(self, party_name):
        total_votes = self.valid_votes
        if party_name in self.parties:
            return self.parties[party_name].get_percentage(total_votes)
        return 0

    def save_to_file(self):
        data = {
            'Constituency Name': self.name,
            'Region': self.region,
            'Country': self.country,
            'Electorate': self.electorate,
            'Valid Votes': self.valid_votes,
            'Invalid Votes': self.invalid_votes,
            'MP': f"{self.mp.first_name} {self.mp.last_name}",
            'Party': self.mp.party,
            'Votes Cast': self.mp.votes_cast
        }
        return data

In [31]:
# Function to read the CSV file with error handling for missing file or incorrect format
def load_data(file_path):
    try:
        return pd.read_csv(file_path, encoding='ISO-8859-1')  # Use 'ISO-8859-1' for compatibility
    except FileNotFoundError:
        print(f"Error: The file {file_path} was not found.")
        exit(1)
    except pd.errors.ParserError:
        print(f"Error: Failed to parse the file {file_path}. Please ensure it's a valid CSV.")
        exit(1)

In [32]:
# Function to process the data into Constituency objects
def process_data(df):
    constituencies = {}
    for index, row in df.iterrows():
        mp_name = f"{row['Member first name']} {row['Member surname']}"
        mp = MP(row['Member first name'], row['Member surname'], row['Member gender'], row['Result'], row['First party'], row['Valid votes'])

        constituency_name = row['Constituency name']
        if constituency_name not in constituencies:
            constituency = Constituency(row['Constituency name'], row['Region name'], row['Country name'], row['Electorate'], row['Valid votes'], row['Invalid votes'], mp)
            constituencies[constituency_name] = constituency
        else:
            constituency = constituencies[constituency_name]

        constituency.add_party_votes(row['First party'], row['Valid votes'])
        if 'Second party' in row and pd.notna(row['Second party']):
            constituency.add_party_votes(row['Second party'], row['Valid votes'])

    return constituencies

In [33]:
# Function to save the data to a CSV file
def save_data_to_csv(constituencies, filename="constituencies_data.csv"):
    try:
        with open(filename, mode='w', newline='') as file:
            writer = csv.DictWriter(file, fieldnames=["Constituency Name", "Region", "Country", "Electorate", "Valid Votes", "Invalid Votes", "MP", "Party", "Votes Cast"])
            writer.writeheader()
            for constituency in constituencies.values():
                writer.writerow(constituency.save_to_file())
        print("Data saved successfully.")
    except Exception as e:
        print(f"Error saving data to file: {e}")


In [34]:
# Function to calculate additional statistics
def calculate_statistics(constituencies):
    total_mps_female = sum(1 for c in constituencies.values() if c.mp.gender == "F")
    total_mps_male = sum(1 for c in constituencies.values() if c.mp.gender == "M")
    total_mps = len(constituencies)
    female_percentage = (total_mps_female / total_mps) * 100 if total_mps > 0 else 0
    male_percentage = (total_mps_male / total_mps) * 100 if total_mps > 0 else 0

    return {
        "Total MPs": total_mps,
        "Female MPs": total_mps_female,
        "Male MPs": total_mps_male,
        "Female Percentage": female_percentage,
        "Male Percentage": male_percentage,
    }

In [35]:
# Function to calculate total votes cast across all constituencies
def get_total_votes_cast(constituencies):
    total_votes = 0
    for constituency in constituencies.values():
        total_votes += constituency.valid_votes  # Add valid votes from each constituency
    return total_votes


In [36]:
# Function to calculate total votes for a given party across all constituencies
def get_party_total_votes_across_constituencies(constituencies, party_name):
    party_total_votes = 0
    for constituency in constituencies.values():
        if party_name in constituency.parties:
            party_total_votes += constituency.parties[party_name].total_votes  # Add votes for the party
    return party_total_votes


In [37]:
# Main function to run the program
def main():
    # Upload the CSV file
    from google.colab import files
    uploaded = files.upload()
    file_name = list(uploaded.keys())[0]

    # Load and process the data
    df = load_data(file_name)
    constituencies = process_data(df)

    while True:
        print("\nVoting Analysis System")
        print("1. View Party Vote Percentage (Across all constituencies)")
        print("2. View Constituency Details")
        print("3. View Candidate Party")
        print("4. View Candidate Name")
        print("5. View Parliamentary Seat Name")
        print("6. View Total Registered Voters")
        print("7. View Total Votes Cast")
        print("8. View Votes Cast for Candidate")
        print("9. Save Data to File")
        print("10. View Statistics")
        print("11. Exit")

        try:
            choice = input("Please enter your choice: ")

            if choice == '1':
                party_name = input("Enter party name: ")
                total_votes = get_total_votes_cast(constituencies)
                party_total_votes = get_party_total_votes_across_constituencies(constituencies, party_name)

                if total_votes > 0:
                    percentage = (party_total_votes / total_votes) * 100
                    print(f"Party {party_name} received {percentage:.2f}% of total votes across all constituencies.")
                else:
                    print("No votes cast across constituencies.")

            elif choice == '2':
                constituency_name = input("Enter constituency name: ")
                if constituency_name in constituencies:
                    constituency = constituencies[constituency_name]
                    mp = constituency.mp
                    print(f"Constituency: {constituency.name}")
                    print(f"Region: {constituency.region}")
                    print(f"Country: {constituency.country}")
                    print(f"Electorate: {constituency.electorate}")
                    print(f"Valid Votes: {constituency.valid_votes}")
                    print(f"Invalid Votes: {constituency.invalid_votes}")
                    print(f"MP: {mp.first_name} {mp.last_name}")
                    print(f"Party: {mp.party}")
                    print(f"Votes Cast: {mp.votes_cast}")
                else:
                    print("Constituency not found.")

            elif choice == '3':
                constituency_name = input("Enter constituency name: ")
                if constituency_name in constituencies:
                    constituency = constituencies[constituency_name]
                    print(f"Candidate Party in {constituency_name}: {constituency.mp.party}")
                else:
                    print("Constituency not found.")

            elif choice == '4':
                constituency_name = input("Enter constituency name: ")
                if constituency_name in constituencies:
                    constituency = constituencies[constituency_name]
                    mp = constituency.mp
                    print(f"Candidate Name in {constituency_name}: {mp.first_name} {mp.last_name}")
                else:
                    print("Constituency not found.")

            elif choice == '5':
                constituency_name = input("Enter constituency name: ")
                if constituency_name in constituencies:
                    constituency = constituencies[constituency_name]
                    print(f"Parliamentary Seat (Constituency) Name: {constituency_name}")
                else:
                    print("Constituency not found.")

            elif choice == '6':
                constituency_name = input("Enter constituency name: ")
                if constituency_name in constituencies:
                    constituency = constituencies[constituency_name]
                    print(f"Total Registered Voters in {constituency_name}: {constituency.electorate}")
                else:
                    print("Constituency not found.")

            elif choice == '7':
                constituency_name = input("Enter constituency name: ")
                if constituency_name in constituencies:
                    constituency = constituencies[constituency_name]
                    print(f"Total Votes Cast in {constituency_name}: {constituency.valid_votes}")
                else:
                    print("Constituency not found.")

            elif choice == '8':
                constituency_name = input("Enter constituency name: ")
                if constituency_name in constituencies:
                    constituency = constituencies[constituency_name]
                    print(f"Votes Cast for Candidate in {constituency_name}: {constituency.mp.votes_cast}")
                else:
                    print("Constituency not found.")

            elif choice == '9':
                print("Saving data...")
                save_data_to_csv(constituencies)

            elif choice == '10':
                statistics = calculate_statistics(constituencies)
                print("Statistics:")
                for key, value in statistics.items():
                    print(f"{key}: {value}")

            elif choice == '11':
                print("Exiting the program.")
                break
            else:
                print("Invalid option. Please try again.")

        except Exception as e:
            print(f"An error occurred: {e}")



In [38]:
# Run the program
if __name__ == "__main__":
    main()


Saving editeddata6368.csv to editeddata6368 (4).csv

Voting Analysis System
1. View Party Vote Percentage (Across all constituencies)
2. View Constituency Details
3. View Candidate Party
4. View Candidate Name
5. View Parliamentary Seat Name
6. View Total Registered Voters
7. View Total Votes Cast
8. View Votes Cast for Candidate
9. Save Data to File
10. View Statistics
11. Exit
Please enter your choice: 1
Enter party name: Lab
Party Lab received 77.56% of total votes across all constituencies.

Voting Analysis System
1. View Party Vote Percentage (Across all constituencies)
2. View Constituency Details
3. View Candidate Party
4. View Candidate Name
5. View Parliamentary Seat Name
6. View Total Registered Voters
7. View Total Votes Cast
8. View Votes Cast for Candidate
9. Save Data to File
10. View Statistics
11. Exit
Please enter your choice: 2
Enter constituency name: Aberafan Maesteg
Constituency: Aberafan Maesteg
Region: Wales
Country: Wales
Electorate: 72580
Valid Votes: 35755
Inv